# 1. Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import warnings
import re
import scipy
import nltk
import pickle
import math
import tensorflow as tf
import random as rn
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
from scipy.sparse import hstack
from tensorflow.keras.models import model_from_json
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer() 
warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# 2. Predefined Functions and variables

In [2]:
!cp /content/drive/My\ Drive/mercari\ data/brands.pkl /content
!cp /content/drive/My\ Drive/mercari\ data/main_cats.pkl /content
!cp /content/drive/My\ Drive/mercari\ data/main_sub_cats.pkl /content
!cp /content/drive/My\ Drive/mercari\ data/sub_cats.pkl /content
!cp /content/drive/My\ Drive/mercari\ data/item_condition_id_vectorizer.pkl /content
!cp /content/drive/My\ Drive/mercari\ data/number_of_colours_vectorizer.pkl /content
!cp /content/drive/My\ Drive/mercari\ data/brand_name_codes_vectorizer.pkl /content
!cp /content/drive/My\ Drive/mercari\ data/main_category_codes_vectorizer.pkl /content
!cp /content/drive/My\ Drive/mercari\ data/main_sub_category_codes_vectorizer.pkl /content
!cp /content/drive/My\ Drive/mercari\ data/sub_category_codes_vectorizer.pkl /content
!cp /content/drive/My\ Drive/mercari\ data/feature_ml.pkl /content
!cp /content/drive/My\ Drive/mercari\ data/finalized_ridge.sav /content
!cp /content/drive/My\ Drive/mercari\ data/finalized_lgbm.sav /content
!cp /content/drive/My\ Drive/mercari\ data/feature_mlp.pkl /content
!cp /content/drive/My\ Drive/mercari\ data/mlp_model.json /content
!cp /content/drive/My\ Drive/mercari\ data/mlp_model.h5 /content

In [3]:
def fill_brands_na(train_brand_name, all_text, all_text_no_punc):
  '''
      This function imputes the missing values for brands in the given dataset
  '''
  merge_brands_more_than_1_word, \
  brands_with_1_word, \
  brands_with_1_word_junk_extra, \
  brands_with_1_word_junk = pickle.load(open('brands.pkl', 'rb'))

  # **********IMPUTATION STARTS HERE**********
  
  for counter in range(len(train_brand_name)):
    #====================================
    # STEP 1
    #====================================

    # merge_brands_more_than_1_word --- text with punctuation
    if train_brand_name[counter] == "Unknown Brand.":
      count = 0
      for brand, preprocessed_brands in merge_brands_more_than_1_word.items():
        for preprocessed_brand in preprocessed_brands:
          if preprocessed_brand in all_text[counter]:
            train_brand_name[counter] = brand
            count = 1
            break
        if count>0:
          break
        
    if train_brand_name[counter] != "Unknown Brand.":
      continue

    # merge_brands_more_than_1_word --- text with no punctuation
    if train_brand_name[counter] == "Unknown Brand.":
      count = 0
      for brand, preprocessed_brands in merge_brands_more_than_1_word.items():
        for preprocessed_brand in preprocessed_brands:
          if preprocessed_brand in all_text_no_punc[counter]:
            train_brand_name[counter] = brand
            count = 1
            break
        if count>0:
          break

    if train_brand_name[counter] != "Unknown Brand.":
      continue

    #====================================
    # STEP 2
    #====================================

    # brands_with_1_word --- text with punctuation
    if train_brand_name[counter] == "Unknown Brand.":
      count = 0
      for brand, preprocessed_brands in brands_with_1_word.items():
        for preprocessed_brand in preprocessed_brands:
          if preprocessed_brand in all_text[counter].split():
            train_brand_name[counter] = brand
            count = 1
            break
        if count>0:
          break

    if train_brand_name[counter] != "Unknown Brand.":
      continue

    #brands_with_1_word --- text with no punctuation
    if train_brand_name[counter] == "Unknown Brand.":
      count = 0
      for brand, preprocessed_brands in brands_with_1_word.items():
        for preprocessed_brand in preprocessed_brands:
          if preprocessed_brand in all_text_no_punc[counter].split():
            train_brand_name[counter] = brand
            count = 1
            break
        if count>0:
          break

    if train_brand_name[counter] != "Unknown Brand.":
      continue

    #====================================
    # STEP 3
    #====================================

    #brands_with_1_word_junk_extra --- text with punctuation
    if train_brand_name[counter] == "Unknown Brand.":
      count = 0
      for brand, preprocessed_brands in brands_with_1_word_junk_extra.items():
        for preprocessed_brand in preprocessed_brands:
          if preprocessed_brand in all_text[counter]:
            train_brand_name[counter] = brand
            count = 1
            break
        if count>0:
          break
        
    if train_brand_name[counter] != "Unknown Brand.":
      continue

    # brands_with_1_word_junk_extra --- text with no punctuation
    if train_brand_name[counter] == "Unknown Brand.":
      count = 0
      for brand, preprocessed_brands in brands_with_1_word_junk_extra.items():
        for preprocessed_brand in preprocessed_brands:
          if preprocessed_brand in all_text_no_punc[counter]:
            train_brand_name[counter] = brand
            count = 1
            break
        if count>0:
          break

    if train_brand_name[counter] != "Unknown Brand.":
      continue

    #====================================
    # STEP 4
    #====================================

    # brands_with_1_word_junk --- text with punctuation
    if train_brand_name[counter] == "Unknown Brand.":
      count = 0
      for brand, preprocessed_brands in brands_with_1_word_junk.items():
        for preprocessed_brand in preprocessed_brands:
          if preprocessed_brand in all_text[counter].split():
            train_brand_name[counter] = brand
            count = 1
            break
        if count>0:
          break

    if train_brand_name[counter] != "Unknown Brand.":
      continue

    #brands_with_1_word_junk --- text with no punctuation
    if train_brand_name[counter] == "Unknown Brand.":
      count = 0
      for brand, preprocessed_brands in brands_with_1_word_junk.items():
        for preprocessed_brand in preprocessed_brands:
          if preprocessed_brand in all_text_no_punc[counter].split():
            train_brand_name[counter] = brand
            count = 1
            break
        if count>0:
          break

  del merge_brands_more_than_1_word
  del brands_with_1_word
  del brands_with_1_word_junk_extra
  del brands_with_1_word_junk

  return train_brand_name

In [4]:
def brand_text(df):
  '''
  This function preprocesses the textual data present in the dataset
  '''
  
  df["item_description"] = df["item_description"].fillna(value="No description yet.")
  all_text = list(df['name']+" "+df['item_description'])

  #define punctuation
  all_text_no_punc = []
  punctuations = '!()-[]{};:\'\"\,<>./?@#$%^&*_~©®™'
  for i in range(len(all_text)):
    no_punct = ""
    for char in all_text[i]:
      if char not in punctuations:
        no_punct = no_punct + char
      else:
        no_punct = no_punct + " "
    #Appending the text version with no punctuations
    all_text_no_punc.append((" ".join(no_punct.split())).lower())
    #Appending the text version with punctuations
    all_text[i] = all_text[i].lower()

  return all_text, all_text_no_punc

In [5]:
def fill_cats(cat_list, cat_string, unique_categories, all_text):
  for i in range(len(cat_list)):
    if cat_list[i] == cat_string:
      count = 0
      for k,v in unique_categories.items():
        for cat in v:
          if cat in all_text[i]:
            cat_list[i] = k
            count = 1
            break
        if count>0:
          break
  return cat_list

def fill_cats_na(X,all_text):
  main_categories = pickle.load(open('main_cats.pkl', 'rb'))
  main_sub_categories = pickle.load(open('main_sub_cats.pkl', 'rb'))
  sub_categories = pickle.load(open('sub_cats.pkl', 'rb'))

  # There are 3 categories per Category Name in the data.
  # The format of the category is : main_category/main_sub_category/sub_category
  # So I have created 3 lists to split them with respect to the punctuation '/'.
  # This is the code to create lists of main and its sub-categories

  categories = list(X['category_name'])
  main_category = []
  main_sub_category = []
  sub_category = []

  for i in categories:
    if not i=="Unknown Category.":
      temp = i.split('/')
      main_category.append(temp[0])
      main_sub_category.append(temp[1])
      sub_category.append(temp[2])
    else:
      main_category.append("Unknown Main-Category.")
      main_sub_category.append("Unknown Main-Sub-Category.")
      sub_category.append("Unknown Sub-Category.")

  main_category = fill_cats(main_category,"Unknown Main-Category.",main_categories,all_text)
  del main_categories
  main_sub_category = fill_cats(main_sub_category,"Unknown Main-Sub-Category.",main_sub_categories,all_text)
  del main_sub_categories
  sub_category = fill_cats(sub_category,"Unknown Sub-Category.",sub_categories,all_text)
  del sub_categories

  X['main_category'] = main_category
  del main_category
  X['main_sub_category'] = main_sub_category
  del main_sub_category
  X['sub_category'] = sub_category
  del sub_category
  del X['category_name']
  return X

In [6]:
def fill_colours(X,all_text):
  colours = ["black",
           "grey",
           "white",
           "beige",
           "red",
           "pink",
           "purple",
           "blue",
           "green",
           "yellow",
           "orange",
           "brown",
           "gold",
           "silver"]

  # Initializing a list containing the number of colours present in the text
  number_of_colours = []

  # Initializing a dictionary where - 
  # keys : The colours
  # values : lists containing whether the colour is present or not
  colours_dictionary = {}

  number_of_colours = list(np.zeros(len(X["name"]),np.int))

  for colour in colours:
    temp = []
    counter = 0
    for a in all_text:
      if colour in a:
        temp.append(1)
        number_of_colours[counter] = number_of_colours[counter] + 1
      else:
        temp.append(0)
      counter = counter +1
    colours_dictionary[colour] = temp

  for k,v in colours_dictionary.items():
    X[k] = v
  X["number_of_colours"] = number_of_colours
  return X

In [7]:
def preprocess_text(all_text):
  '''
  This function preprocesses the item description and name of the datasets
  '''

  for i in range(len(all_text)):
    #Preprocessing the words with apostrophes and new line characters as well
    all_text[i] = re.sub(r"won't", "will not", all_text[i])
    all_text[i] = re.sub(r"can\'t", "can not", all_text[i])
    all_text[i] = re.sub(r"n\'t", " not", all_text[i])
    all_text[i] = re.sub(r"\'re", " are", all_text[i])
    all_text[i] = re.sub(r"\'s", " is", all_text[i])
    all_text[i] = re.sub(r"\'d", " would", all_text[i])
    all_text[i] = re.sub(r"\'ll", " will", all_text[i])
    all_text[i] = re.sub(r"\'t", " not", all_text[i])
    all_text[i] = re.sub(r"\'ve", " have", all_text[i])
    all_text[i] = re.sub(r"\'m", " am", all_text[i])
    all_text[i] = all_text[i].replace('\\r', ' ')
    all_text[i] = all_text[i].replace('\\n', ' ')
    all_text[i] = all_text[i].replace('\\"', ' ')
    #Getting rid of unnecessary spaces
    #all_text[i] = ' '.join(e.lower() for e in all_text[i].split() if e.lower() not in stopwords)
    #Keeping only the alphanumeric characters and discarding the rest
    all_text[i] = re.sub('[^A-Za-z0-9 ]', ' ', all_text[i])
    #Getting rid of unnecessary spaces
    #all_text[i] = ' '.join(e.lower() for e in all_text[i].split() if e.lower() not in stopwords)
    all_text[i] = ' '.join(e.lower() for e in all_text[i].split())
    # Converting "24ml" to "24 ml" or even "2v2" to '2 v 2' or many combinations as well
    all_text[i] = re.sub('(?<=[A-Za-z])(?=[0-9])|(?<=[0-9])(?=[A-Za-z])',' ', all_text[i])
    
    #The code below lemmatizes the text wherein inherited texts of a word in substituted with the parent word.
    #For example, go, gone, went and goes will be substituted with the parent word go.
    # I have lemmatized both nouns as well as verbs for that matter
    
    # n denotes "NOUN" in "pos"
    all_text[i] = ' '.join(lemmatizer.lemmatize(i, pos ="n") for i in all_text[i].split())
    # v denotes "VERB" in "pos"
    all_text[i] = ' '.join(lemmatizer.lemmatize(i, pos ="v") for i in all_text[i].split())
  
  return all_text

In [8]:
def rmsle_score(actual, predicted, shape):
  if shape>1:
    return np.sqrt(np.nansum(np.square(np.log(predicted+1) - np.log(actual+1)))/float(len(actual)))
  else:
    return np.sqrt(np.nansum(np.square(np.log(predicted+1) - np.log(actual+1)))/float(1.0))

def rmsle(y_true, y_pred):
    return tf.py_function(rmsle_score, (y_true, y_pred), tf.double)

# 3. Function_1

In [9]:
def function_1(X):
  #Fill NaN values
  X['brand_name'] = X['brand_name'].fillna("Unknown Brand.")
  X["item_description"] = X["item_description"].fillna(value="No description yet.")
  all_text, all_text_no_punc = brand_text(X)
  X['brand_name'] = fill_brands_na(list(X['brand_name']),all_text, all_text_no_punc)
  del all_text_no_punc

  X['category_name'] = X['category_name'].fillna(value="Unknown Category.")
  X = fill_cats_na(X,all_text)
  X = fill_colours(X,all_text)

  item_desc_len = X["item_description"].str.len() 
  X["item_desc_len_octa"] = np.power(item_desc_len,0.125)
  del item_desc_len
  item_name_len = X["name"].str.len()
  X["item_name_len_sqrt"] = np.sqrt(max(item_name_len+1)-item_name_len)
  del item_name_len

  # Creating a boolean column where 1 stands for [rm] is present in the description and 0 for the ones which don't have it.
  item_rm = []
  price_1 = []
  price_0 = []

  for i in range(len(list(X["item_description"]))):
    if '[rm]' in list(X["item_description"])[i]:
      item_rm.append(1)
    else:
      item_rm.append(0)
      
  X["Item_desc_[rm]_flag"] = item_rm

  # Creating a boolean column where 1 stands for [rm] is present in the description and 0 for the ones which don't have it.
  item_rm = []
  price_1 = []
  price_0 = []
  # Replacing NULL values with a dummy string

  for i in range(len(list(X["name"]))):
    if '[rm]' in list(X["name"])[i]:
      item_rm.append(1)
    else:
      item_rm.append(0)

  X["Name_[rm]_flag"] = item_rm
  del item_rm, price_1, price_0
  X['preprocessed_text'] = preprocess_text(all_text)
  del X['name'],X['item_description']

  column = list(X['item_condition_id'])

  for i in range(len(column)):
    if column[i]==1:
      column[i] = 'a'
    elif column[i]==2:
      column[i] = 'b'
    elif column[i]==3:
      column[i] = 'c'
    elif column[i]==4:
      column[i] = 'd'
    else:
      column[i] = 'e'
  X['item_condition_id'] = column
  del column

  colours = scipy.sparse.csr_matrix(X[['black', 'grey', 'white', 'beige', 
                                               'red', 'pink', 'purple', 'blue', 
                                               'green', 'yellow', 'orange', 'brown', 
                                               'gold', 'silver','Item_desc_[rm]_flag', 
                                               'Name_[rm]_flag','item_desc_len_octa',
                                               'item_name_len_sqrt','shipping']].values)
  X['number_of_colours'] = X['number_of_colours'].astype('string')

  item_condition_id_vectorizer = pickle.load(open('item_condition_id_vectorizer.pkl', 'rb'))
  item_condition_id = item_condition_id_vectorizer.transform(X['item_condition_id'].values)
  del item_condition_id_vectorizer
  number_of_colours_vectorizer = pickle.load(open('number_of_colours_vectorizer.pkl', 'rb'))
  number_of_colours = number_of_colours_vectorizer.transform(X['number_of_colours'].values)
  del number_of_colours_vectorizer
  brand_name_codes_vectorizer = pickle.load(open('brand_name_codes_vectorizer.pkl', 'rb'))
  brand_name_codes = brand_name_codes_vectorizer.transform(X['brand_name'].values)
  del brand_name_codes_vectorizer
  main_category_codes_vectorizer = pickle.load(open('main_category_codes_vectorizer.pkl', 'rb'))
  main_category_codes = main_category_codes_vectorizer.transform(X['main_category'].values)
  del main_category_codes_vectorizer
  main_sub_category_codes_vectorizer = pickle.load(open('main_sub_category_codes_vectorizer.pkl', 'rb'))
  main_sub_category_codes = main_sub_category_codes_vectorizer.transform(X['main_sub_category'].values)
  del main_sub_category_codes_vectorizer
  sub_category_codes_vectorizer = pickle.load(open('sub_category_codes_vectorizer.pkl', 'rb'))
  sub_category_codes = sub_category_codes_vectorizer.transform(X['sub_category'].values)
  del sub_category_codes_vectorizer

  final_data = hstack((item_condition_id,
                       number_of_colours,
                       brand_name_codes,
                       main_category_codes,
                       main_sub_category_codes,
                       sub_category_codes,
                       colours)).tocsr()

  vectorizer_tfidf = pickle.load(open('feature_ml.pkl', 'rb'))
  X_tfidf = vectorizer_tfidf.transform(X['preprocessed_text'])
  del vectorizer_tfidf
  X_ml = hstack((final_data,X_tfidf)).tocsr().astype('float32')
  del X_tfidf

  filename = 'finalized_ridge.sav'
  loaded_model = pickle.load(open(filename, 'rb'))
  result_1 = np.power(loaded_model.predict(X_ml),8)
  filename = 'finalized_lgbm.sav'
  loaded_model = pickle.load(open(filename, 'rb'))
  result_2 = np.power(loaded_model.predict(X_ml),8)
  del filename, loaded_model, X_ml

  vectorizer_tfidf = pickle.load(open('feature_mlp.pkl', 'rb'))
  X_tfidf = vectorizer_tfidf.transform(X['preprocessed_text'])
  del vectorizer_tfidf
  X = hstack((final_data,X_tfidf)).tocsr().astype('float32')  
  del X_tfidf
  # load json and create model
  json_file = open("mlp_model.json", 'r')
  loaded_model_json = json_file.read()
  json_file.close()
  loaded_model = model_from_json(loaded_model_json)
  # load weights into new model
  loaded_model.load_weights("mlp_model.h5")
  # evaluate loaded model on test data
  loaded_model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.0005),
                loss=tf.keras.losses.MeanSquaredLogarithmicError(),
                metrics = [rmsle])
  result_3 = loaded_model.predict(X)
  temp = []
  for i in result_3:
    for j in i:
      temp.append(j)
  result_3 = np.array(temp)
  del X,temp
  del json_file, loaded_model_json, loaded_model
  result = 0.1*result_1 + 0.3*result_2 + 0.6*result_3
  return result

In [10]:
train = pd.read_csv("/content/drive/My Drive/mercari data/train.csv")
del train['Unnamed: 0']
price = train['price']
del train['price']

In [11]:
%%time
print("\nPrediction of a single datapoint : ",function_1(train.iloc[[100000]])[0])


Prediction of a single datapoint :  17.9521990680505
CPU times: user 45.1 s, sys: 5.01 s, total: 50.1 s
Wall time: 54.2 s


In [12]:
print("\nActual value : ",price[100000])


Actual value :  19.0


In [13]:
%%time
tf.keras.backend.clear_session()
np.random.seed(0)
rn.seed(0)
print("\nPrediction of multiple datapoints : ",list(function_1(train.iloc[100000:100500])))


Prediction of multiple datapoints :  [15.723630098406936, 18.570566115572113, 52.86479554509762, 15.264150900916919, 5.83399450697756, 21.71636086295865, 4.815199382878427, 27.812748112236243, 12.526355985934131, 40.83565809653654, 23.976273918726065, 11.798341966552567, 70.39023727366724, 18.028935223261108, 159.65747718109853, 14.874348538856427, 9.22598601951536, 18.410751286832998, 38.79286853098621, 32.204333404556905, 12.92167618535252, 13.00588150244323, 13.47370397417669, 93.64618706360393, 22.893931985292532, 15.333665019684705, 40.23675827694302, 10.175613112682617, 28.041719062353394, 21.84405511758862, 61.25062932255206, 20.6276128345548, 19.251453468931544, 415.8492517752627, 16.91747060248891, 17.44840898314644, 22.95421783673899, 32.585048552199375, 46.903517965680756, 12.064505930975796, 12.285217612675948, 31.007755968968986, 30.232572299260063, 19.307005135219846, 16.406651707987827, 12.89070148000846, 16.27671647714855, 31.894940325798842, 29.403420566306817, 28.066

In [14]:
print("\nActual values : ",list(price[100500:100500]))


Actual values :  []


# 4. Function_2

In [15]:
def function_1(X,y):
  #Fill NaN values
  shape = X.shape[0]
  X['brand_name'] = X['brand_name'].fillna("Unknown Brand.")
  X["item_description"] = X["item_description"].fillna(value="No description yet.")
  all_text, all_text_no_punc = brand_text(X)
  X['brand_name'] = fill_brands_na(list(X['brand_name']),all_text, all_text_no_punc)
  del all_text_no_punc

  X['category_name'] = X['category_name'].fillna(value="Unknown Category.")
  X = fill_cats_na(X,all_text)
  X = fill_colours(X,all_text)

  item_desc_len = X["item_description"].str.len() 
  X["item_desc_len_octa"] = np.power(item_desc_len,0.125)
  del item_desc_len
  item_name_len = X["name"].str.len()
  X["item_name_len_sqrt"] = np.sqrt(max(item_name_len+1)-item_name_len)
  del item_name_len

  # Creating a boolean column where 1 stands for [rm] is present in the description and 0 for the ones which don't have it.
  item_rm = []
  price_1 = []
  price_0 = []

  for i in range(len(list(X["item_description"]))):
    if '[rm]' in list(X["item_description"])[i]:
      item_rm.append(1)
    else:
      item_rm.append(0)
      
  X["Item_desc_[rm]_flag"] = item_rm

  # Creating a boolean column where 1 stands for [rm] is present in the description and 0 for the ones which don't have it.
  item_rm = []
  price_1 = []
  price_0 = []
  # Replacing NULL values with a dummy string

  for i in range(len(list(X["name"]))):
    if '[rm]' in list(X["name"])[i]:
      item_rm.append(1)
    else:
      item_rm.append(0)

  X["Name_[rm]_flag"] = item_rm
  del item_rm, price_1, price_0
  X['preprocessed_text'] = preprocess_text(all_text)
  del X['name'],X['item_description']

  column = list(X['item_condition_id'])

  for i in range(len(column)):
    if column[i]==1:
      column[i] = 'a'
    elif column[i]==2:
      column[i] = 'b'
    elif column[i]==3:
      column[i] = 'c'
    elif column[i]==4:
      column[i] = 'd'
    else:
      column[i] = 'e'
  X['item_condition_id'] = column
  del column

  colours = scipy.sparse.csr_matrix(X[['black', 'grey', 'white', 'beige', 
                                               'red', 'pink', 'purple', 'blue', 
                                               'green', 'yellow', 'orange', 'brown', 
                                               'gold', 'silver','Item_desc_[rm]_flag', 
                                               'Name_[rm]_flag','item_desc_len_octa',
                                               'item_name_len_sqrt','shipping']].values)
  X['number_of_colours'] = X['number_of_colours'].astype('string')

  item_condition_id_vectorizer = pickle.load(open('item_condition_id_vectorizer.pkl', 'rb'))
  item_condition_id = item_condition_id_vectorizer.transform(X['item_condition_id'].values)
  del item_condition_id_vectorizer
  number_of_colours_vectorizer = pickle.load(open('number_of_colours_vectorizer.pkl', 'rb'))
  number_of_colours = number_of_colours_vectorizer.transform(X['number_of_colours'].values)
  del number_of_colours_vectorizer
  brand_name_codes_vectorizer = pickle.load(open('brand_name_codes_vectorizer.pkl', 'rb'))
  brand_name_codes = brand_name_codes_vectorizer.transform(X['brand_name'].values)
  del brand_name_codes_vectorizer
  main_category_codes_vectorizer = pickle.load(open('main_category_codes_vectorizer.pkl', 'rb'))
  main_category_codes = main_category_codes_vectorizer.transform(X['main_category'].values)
  del main_category_codes_vectorizer
  main_sub_category_codes_vectorizer = pickle.load(open('main_sub_category_codes_vectorizer.pkl', 'rb'))
  main_sub_category_codes = main_sub_category_codes_vectorizer.transform(X['main_sub_category'].values)
  del main_sub_category_codes_vectorizer
  sub_category_codes_vectorizer = pickle.load(open('sub_category_codes_vectorizer.pkl', 'rb'))
  sub_category_codes = sub_category_codes_vectorizer.transform(X['sub_category'].values)
  del sub_category_codes_vectorizer

  final_data = hstack((item_condition_id,
                       number_of_colours,
                       brand_name_codes,
                       main_category_codes,
                       main_sub_category_codes,
                       sub_category_codes,
                       colours)).tocsr()

  vectorizer_tfidf = pickle.load(open('feature_ml.pkl', 'rb'))
  X_tfidf = vectorizer_tfidf.transform(X['preprocessed_text'])
  del vectorizer_tfidf
  X_ml = hstack((final_data,X_tfidf)).tocsr().astype('float32')
  del X_tfidf

  filename = 'finalized_ridge.sav'
  loaded_model = pickle.load(open(filename, 'rb'))
  result_1 = np.power(loaded_model.predict(X_ml),8)
  filename = 'finalized_lgbm.sav'
  loaded_model = pickle.load(open(filename, 'rb'))
  result_2 = np.power(loaded_model.predict(X_ml),8)
  del filename, loaded_model, X_ml

  vectorizer_tfidf = pickle.load(open('feature_mlp.pkl', 'rb'))
  X_tfidf = vectorizer_tfidf.transform(X['preprocessed_text'])
  del vectorizer_tfidf
  X = hstack((final_data,X_tfidf)).tocsr().astype('float32')  
  del X_tfidf
  # load json and create model
  json_file = open("mlp_model.json", 'r')
  loaded_model_json = json_file.read()
  json_file.close()
  loaded_model = model_from_json(loaded_model_json)
  # load weights into new model
  loaded_model.load_weights("mlp_model.h5")
  # evaluate loaded model on test data
  loaded_model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.0005),
                loss=tf.keras.losses.MeanSquaredLogarithmicError(),
                metrics = [rmsle])
  result_3 = loaded_model.predict(X)
  temp = []
  for i in result_3:
    for j in i:
      temp.append(j)
  result_3 = np.array(temp)
  del X,temp
  del json_file, loaded_model_json, loaded_model
  result = 0.1*result_1 + 0.3*result_2 + 0.6*result_3
  score = rmsle_score(y,result,shape)
  return score

In [16]:
%%time
print("Score of a single datapoint : ",function_1(train.iloc[[100000]],price[100000]))

Score of a single datapoint :  0.053812403590904
CPU times: user 44.1 s, sys: 3.99 s, total: 48.1 s
Wall time: 55.9 s


In [17]:
%%time
print("\nScore of multiple predictions : ",function_1(train.iloc[100000:100500],price[100000:100500]))


Score of multiple predictions :  0.3805815153812488
CPU times: user 45.3 s, sys: 4.5 s, total: 49.8 s
Wall time: 49.4 s
